# **Models**

The various models learned in class were implemented to attempt prediction of the wearer's arm movements based upon the output of the six sensors embedded in the shirt. The data used in these models were from a single subject, pooled across all three trials collected in the protocol. The subject data was taken from subject #4, because that particular data set had the fewest outliers as discussed in the **Data Description and Initial EDA** section. Additionally, due to the high frequency of the collecte data, the data was downsampled to reduce the training time of the models.




**Train Test Split**

The data was split as a function of time - with the first 80% of the data used as the training set, and the last 20% reserved for testing. Initially, we had done a train_test_split randomly, however this yields artificially inflated test scores of the models because in the actual application of the shirt, it is impossible to predict\results based on future data.

## **A. Data Preparation and Cleaning**

In [23]:
plt.style.use('ac209a.mplstyle')

In [24]:
def standardize(X, X_ref):
    mean = np.mean(X_ref)
    std = np.std(X_ref)
    return (X-mean)/std

In [25]:
def normalize(X, X_ref):
    return (X-np.min(X_ref))/(np.max(X_ref)-np.min(X_ref))

In [26]:
def deriv_1st(df, features):
    prev = df.iloc[0:-2]
    nxt = df.iloc[2:]
    new_features = [f+'_1st' for f in features]
    results = pd.DataFrame(nxt[features].values - prev[features].values, columns=new_features)
    time_diff = pd.DataFrame(nxt['t'].values-prev['t'].values)
    for feature in new_features:
        results[feature] = results[[feature]].div(time_diff.values)
    return results

In [27]:
def deriv_2nd(df, features):
    prev = df.iloc[0:-2]
    now = df.iloc[1:-1]
    nxt = df.iloc[2:]
    new_features = [f+'_2nd' for f in features]
    results = pd.DataFrame(nxt[features].values-2*now[features].values+prev[features].values, columns=new_features)
    time_diff = pd.DataFrame((nxt['t'].values-now['t'].values)*(now['t'].values-prev['t'].values))
    for feature in new_features:
        results[feature] = results[[feature]].div(time_diff.values)
    return results

In [28]:
def prev_values(df, features, num):
    for i in range(1, num+1):
        new_features = [f+'_prev_'+str(i) for f in features]
        prev_df = df[features].shift(i)
        prev_df.columns = new_features
        df = pd.concat([df, prev_df], axis=1)
    return df.dropna()

In [29]:
def add_features(df, features, angs, if_1st_deriv=False, if_2nd_deriv=False, num_prev=0):
    if if_1st_deriv: 
        df_1st_deriv = deriv_1st(df, features)
    else:
        df_1st_deriv = pd.DataFrame()
    if if_2nd_deriv:
        df_2nd_deriv = deriv_2nd(df, features)
    else:
        df_2nd_deriv = pd.DataFrame()
    results = pd.concat([pd.DataFrame(df[1:-1].values, columns=df.columns), df_1st_deriv, df_2nd_deriv], axis=1)
    results = prev_values(results, features, num_prev).drop('t', axis=1)
    return results.drop(columns=angs), results[angs]

In [31]:
def load_data(names, nums, tests, features, angs, if_1st_deriv=False, if_2nd_deriv=False, num_prev=0):
    X_list = []
    y_list = []

    for number in nums:
        for test in tests:
            for name in names:
                file_dir = 'data/' + name + '_t' + str(number) + test + '.txt'
                data = pd.read_csv(file_dir, header=None,
                                   names=['t','s1','s2','s3','s4','s5','s6','hf','ab','ir'])
                data = data.iloc[::20]
                X_df, y_df = add_features(data, features, angs, if_1st_deriv=if_1st_deriv, if_2nd_deriv=if_2nd_deriv, num_prev=num_prev)
                X_list.append(X_df)
                y_list.append(y_df)
                
    X_return = pd.concat(X_list)
    y_return = pd.concat(y_list)
    return X_return, y_return

In [32]:
def calc_plot_arrays(y, yhat):
    x_list = []
    num_list = []
    mean_list = []
    std_list = []
    mae_list = []
    rmse_list = []
    
    for val in np.arange(math.floor(min(y)), math.ceil(max(y))):
        idx = (y>val)&(y<val+1)
        y_pred = np.array(yhat)[np.where(idx)]
        y_true = np.array(y)[np.where(idx)]
        if len(y_true) > 0:
            x_list.append(np.mean(y_true))
            num_list.append(len(y_true))
            mean_list.append(np.mean(y_pred))
            std_list.append(np.std(y_pred))
            mae_list.append(mean_absolute_error(y_true=y_true, y_pred=y_pred))
            rmse_list.append(np.sqrt(mean_squared_error(y_true=y_true, y_pred=y_pred)))
        
    plot_dict = {'x_arr': x_list, 'num_arr':num_list, 'mean_arr':mean_list,
                 'std_arr': std_list, 'mae_arr':mae_list, 'rmse_arr':rmse_list}
    
    return plot_dict


In [33]:
def display_result(model, X_train, y_train, X_test, y_test, plot=False):
    yhat_train = model.predict(X_train)
    yhat_test = model.predict(X_test)

    r2_train = r2_score(y_train, yhat_train)
    r2_test = r2_score(y_test, yhat_test)
    
    rmse_train = np.sqrt(mean_squared_error(y_true=y_train, y_pred=yhat_train))
    rmse_test = np.sqrt(mean_squared_error(y_true=y_test, y_pred=yhat_test))
    
    mae_train = mean_absolute_error(y_true=y_train, y_pred=yhat_train)
    mae_test = mean_absolute_error(y_true=y_test, y_pred=yhat_test)
    
    plot_dict_train = calc_plot_arrays(y_train, yhat_train)
    plot_dict_test = calc_plot_arrays(y_test, yhat_test)
    
    mean_std_train = np.mean(plot_dict_train['std_arr'])
    mean_std_test = np.mean(plot_dict_test['std_arr'])
    
    dict_train = {'r2':r2_train, 'mean_std':mean_std_train, 'rmse':rmse_train, 'mae':mae_train}
    dict_test = {'r2':r2_test, 'mean_std':mean_std_test, 'rmse':rmse_test, 'mae':mae_test}
    
    if plot:
        fig, axs = plt.subplots(2,2, figsize=(30,20))
        for case in range(0,2):
            if case == 0: 
                plot_dict = plot_dict_train
                plt_y = y_train
                title = 'Train'
            else: 
                plot_dict = plot_dict_test
                plt_y = y_test
                title = 'Test'

            plt_x = plot_dict['x_arr']
            plt_mean = np.array(plot_dict['mean_arr'])
            plt_std = np.array(plot_dict['std_arr'])

            axs[0,case].plot(plt_x, plt_mean)
            axs[0,case].fill_between(plt_x, plt_mean-plt_std, plt_mean+plt_std, alpha=0.3)
            axs[0,case].plot([min(plt_y), max(plt_y)], [min(plt_y), max(plt_y)], '--')
            axs[0,case].set_xlabel('True Angles')
            axs[0,case].set_ylabel('Predicted Angles')
            axs[0,case].set_title(title)

            axs[1,case].plot(plt_x, plot_dict['rmse_arr'], label='RMSE')
            axs[1,case].plot(plt_x, plot_dict['mae_arr'], label='MAE')
            axs[1,case].set_xlabel('True Angles')
            axs[1,case].set_ylabel('Error')
            axs[1,case].set_title(title)
            axs[1,case].legend()
        plt.show()
    result_df = pd.DataFrame([dict_train, dict_test], index=['Train','Test'])[['r2','mean_std','mae','rmse']]
    return result_df, dict_train, dict_test, plot_dict_train, plot_dict_test
    

In [149]:
def display_all_results(model, X_df, df, if_display=True, if_plot=False, 
                        if_stand=False, if_norm=False,
                        if_PCA=False, poly_deg=1):
    frames = []
    sens = ['s'+str(k) for k in range(1,n_sens+1)]
    angs = ['ab', 'hf', 'ir']
    models = []
    poly = PolynomialFeatures(degree=poly_deg)
    
    X_train, X_test, y_train, y_test = train_test_split(X_df, df[angs], test_size=0.2, shuffle=False) #, random_state=42)
        
    if if_stand: 
        X_test = standardize(X_test, X_train)
        X_train = standardize(X_train, X_train)

    if if_norm:
        X_test = normalize(X_test, X_train)
        X_train = normalize(X_train, X_train)
                
    if poly_deg > 1:
        X_train_add = pd.DataFrame(poly.fit_transform(X_train[sens]))
        X_test_add = pd.DataFrame(poly.fit_transform(X_test[sens]))
        
        X_train = pd.concat([pd.DataFrame(X_train.values, columns=X_train.columns), 
                             X_train_add], axis=1).drop(sens, axis=1)
        X_test = pd.concat([pd.DataFrame(X_test.values, columns=X_test.columns), 
                            X_test_add], axis=1).drop(sens, axis=1)
    
    if if_PCA:
        # Note: should normalize before applying PCA
        n = X_train.shape[1]
        pca = PCA(n).fit(X_train)
        i = 1
        while np.sum(pca.explained_variance_ratio_[:i]) < 0.95:
            i += 1
        print([n, i])
        pca = PCA(i).fit(X_train)
        X_train = pca.transform(X_train)
        X_test = pca.transform(X_test)
    
    for i,a in enumerate(angs):
        model.fit(X_train, y_train[a])
        if if_plot == True: print(f'\n********** {a}: **********\n')
        results = display_result(model, X_train, y_train[a], X_test, y_test[a], plot=if_plot)
        frames.append(results[0])
        models.append(model)
        
    if if_display == True: display(pd.concat(frames, keys=angs))
        
    return frames, models
    

In [35]:
def find_best_param(scores_list, params, param_name):
    for ind, score in enumerate(scores_list):
        best_score = np.max(score)
        best_ind = np.argmax(score)
        best_param = params[best_ind]
        print(angs[ind] + ':')
        print("The best model has " + param_name + " of {}.".format(best_param))                                                                      
        print("The best model has R2 = {}.\n".format(best_score))

In [196]:
n_sens = 6
sens = ['s'+str(k) for k in range(1,n_sens+1)]
angs = ['ab', 'hf', 'ir']

X_df, y_df = load_data(['M1'], [1,2,3], ['A','B','C'], angs=angs, features=sens)
X_df_added, y_df_added = load_data(['M1'], [1,2,3], ['A','B','C'],
                                   angs=angs, features=sens, 
                                   if_1st_deriv=True, if_2nd_deriv=True, num_prev=2)

## **B. Linear Regression**

In [37]:
linreg = LinearRegression()
display_all_results(linreg, X_df, y_df);

r2   mean_std        mae       rmse
ab Train  0.901010  11.897278   9.753486  12.761176
   Test   0.775065  12.240685  14.914385  19.012592
hf Train  0.752430  14.254330  16.145868  19.993084
   Test   0.602819  14.183518  19.324524  23.550622
ir Train  0.591279  15.805040  21.354235  26.978226
   Test   0.544879  11.211689  17.111802  21.717219

In [38]:
linreg = LinearRegression()
display_all_results(linreg, X_df_added, y_df_added);

r2   mean_std        mae       rmse
ab Train  0.911068  11.183976   9.135420  12.087205
   Test   0.779129  11.777138  14.690716  18.811488
hf Train  0.809676  13.772353  13.929124  17.546370
   Test   0.616968  14.639168  18.787604  23.138092
ir Train  0.649530  15.648047  20.272851  24.967519
   Test   0.586149  11.540126  16.253234  20.613524

Top is six sensor input. Bottom is six sensor with prev vals and derivatives as input. We see improvement on the bottom one. We are gonna use data set with additional features. 

**C. Polynomial Regression**

In [155]:
degrees = range(2,7)
ab_scores, hf_scores, ir_scores = [], [], []
linreg = LinearRegression()

for deg in degrees:
    print('\nDegree = ', deg)
    result,_ = display_all_results(linreg, X_df_added, y_df_added, poly_deg=deg);
    ab_scores.append(result[0].r2[1])
    hf_scores.append(result[1].r2[1])
    ir_scores.append(result[2].r2[1])

scores_list = [ab_scores, hf_scores, ir_scores]
find_best_param(scores_list, degrees, 'degree')


Degree =  2


r2   mean_std        mae       rmse
ab Train  0.928443  10.310230   8.034766  10.842308
   Test   0.790201  11.929353  14.167058  18.333891
hf Train  0.853046  12.920511  12.166673  15.418108
   Test   0.656704  14.182903  17.672000  21.905059
ir Train  0.742417  15.765526  16.901706  21.404681
   Test   0.646182  12.014861  15.041473  19.059892


Degree =  3


r2   mean_std        mae       rmse
ab Train  0.938691   9.631045   7.223615  10.035931
   Test   0.805583  12.408493  13.439350  17.649004
hf Train  0.883283  11.418579  10.591613  13.740668
   Test   0.694339  14.135169  16.298434  20.669499
ir Train  0.792263  15.181595  14.827330  19.222370
   Test   0.711362  11.543475  13.234511  17.215008


Degree =  4


...


Degree =  5


r2   mean_std        mae       rmse
ab Train  0.956887   8.149036   5.854546   8.415870
   Test   0.779042  13.934190  14.370312  18.815160
hf Train  0.925327   9.277708   8.195403  10.990606
   Test   0.649857  15.716721  17.273981  22.122440
ir Train  0.888803  11.585649  10.484527  14.063628
   Test   0.477850  16.328787  16.566853  23.154092


Degree =  6


r2   mean_std        mae       rmse
ab Train  0.970348   6.750935   4.792580   6.979420
   Test   0.335660  22.612346  20.522561  32.624865
hf Train  0.950048   7.693948   6.526878   8.989149
   Test  -0.072901  27.886545  24.026781  38.724874
ir Train  0.931422   9.214360   7.859736  11.044416
   Test  -1.245714  28.648175  26.246970  48.018375

ab:
The best model has degree of 3.
The best model has R2 = 0.8055832511619616.

hf:
The best model has degree of 3.
The best model has R2 = 0.6943393775038769.

ir:
The best model has degree of 3.
The best model has R2 = 0.7113616735074386.



## **D. kNN**

In [42]:
k_values = [1, 10, 75, 250, 500, 750, 1000]
ab_scores, hf_scores, ir_scores = [], [], []

for val in k_values:
    print('\nk =', val)
    kNN = KNeighborsRegressor(n_neighbors=val)
    result,_ = display_all_results(kNN, X_df, y_df)
    ab_scores.append(result[0].r2[1])
    hf_scores.append(result[1].r2[1])
    ir_scores.append(result[2].r2[1])

scores_list = [ab_scores, hf_scores, ir_scores]
find_best_param(scores_list, k_values, 'k')


k = 1


r2   mean_std        mae       rmse
ab Train  1.000000   0.270669   0.000000   0.000000
   Test   0.678463  17.175134  16.561574  22.731489
hf Train  1.000000   0.270279   0.000000   0.000000
   Test   0.469556  19.566608  20.546974  27.216222
ir Train  1.000000   0.259164   0.000000   0.000000
   Test   0.467840  17.389108  16.620756  23.483450


k = 10


r2   mean_std        mae       rmse
ab Train  0.945282   9.230150   6.561840   9.487675
   Test   0.778792  13.967837  14.266087  18.854402
hf Train  0.899476  10.673805   9.362343  12.739906
   Test   0.623361  15.636259  17.447701  22.933512
ir Train  0.867125  11.748420  10.733435  15.382324
   Test   0.674155  13.459029  13.214304  18.375810


k = 75


...


k = 750


r2   mean_std        mae       rmse
ab Train  0.891408   9.783162   9.989823  13.365741
   Test   0.772741  11.785113  14.866803  19.110529
hf Train  0.721782  10.175498  17.197446  21.194503
   Test   0.619432  11.555365  18.640482  23.052836
ir Train  0.615571  13.863966  20.320217  26.164238
   Test   0.560100   9.803252  16.645006  21.350983


k = 1000


r2   mean_std        mae       rmse
ab Train  0.874994   9.264391  10.715677  14.340357
   Test   0.762510  11.040825  15.219314  19.535970
hf Train  0.684521   9.663783  18.372488  22.569206
   Test   0.590854  10.738913  19.466078  23.902706
ir Train  0.590661  13.594430  21.091222  26.998599
   Test   0.533499   9.319936  17.140564  21.987053

ab:
The best model has k of 75.
The best model has R2 = 0.7872503962714331.

hf:
The best model has k of 75.
The best model has R2 = 0.6604305914334088.

ir:
The best model has k of 10.
The best model has R2 = 0.6741549638511204.



In [43]:
k_values = [1, 10, 75, 250, 500, 750, 1000]
ab_scores, hf_scores, ir_scores = [], [], []

for val in k_values:
    print('\nk =', val)
    kNN = KNeighborsRegressor(n_neighbors=val)
    result,_ = display_all_results(kNN, X_df_added, y_df_added)
    ab_scores.append(result[0].r2[1])
    hf_scores.append(result[1].r2[1])
    ir_scores.append(result[2].r2[1])

scores_list = [ab_scores, hf_scores, ir_scores]
find_best_param(scores_list, k_values, 'k')


k = 1


r2   mean_std        mae       rmse
ab Train  1.000000   0.270728   0.000000   0.000000
   Test   0.663801  17.926115  16.186274  23.208725
hf Train  1.000000   0.270308   0.000000   0.000000
   Test   0.588078  17.520518  17.158687  23.994822
ir Train  1.000000   0.259107   0.000000   0.000000
   Test   0.380926  16.583154  17.703968  25.211659


k = 10


r2   mean_std        mae       rmse
ab Train  0.957935   7.951207   5.760661   8.312949
   Test   0.774396  14.262204  13.842188  19.011956
hf Train  0.933720   8.603562   7.473064  10.354538
   Test   0.699214  14.918886  15.272554  20.504022
ir Train  0.906927   9.894820   9.307621  12.866523
   Test   0.585610  12.343559  15.461013  20.626941


k = 75


...


k = 750


r2   mean_std        mae       rmse
ab Train  0.808628   9.534601  13.473515  17.731061
   Test   0.639734  10.808876  18.673569  24.025096
hf Train  0.569033   8.227947  21.356790  26.403573
   Test   0.415114   8.250934  23.186845  28.592077
ir Train  0.526811  11.892103  23.778674  29.011307
   Test   0.270223   8.298706  22.390715  27.373176


k = 1000


r2   mean_std        mae       rmse
ab Train  0.767586   9.144878  15.103589  19.540088
   Test   0.588936  10.095918  20.147723  25.663025
hf Train  0.514977   7.378218  22.771343  28.010580
   Test   0.359721   7.321765  24.330087  29.915394
ir Train  0.475553  11.228812  25.419798  30.542256
   Test   0.206303   7.697991  23.920237  28.546811

ab:
The best model has k of 10.
The best model has R2 = 0.7743959737106147.

hf:
The best model has k of 10.
The best model has R2 = 0.699213948625646.

ir:
The best model has k of 10.
The best model has R2 = 0.5856100881336894.



## **E. Ridge**

In [44]:
lambdas = [.01,.05,.1,.5,1,5,10,50,100]
degrees = np.arange(1,6)
ab_ridge_scores = np.zeros((len(lambdas),len(degrees)))
hf_ridge_scores = np.zeros((len(lambdas),len(degrees)))
ir_ridge_scores = np.zeros((len(lambdas),len(degrees)))

for ind1, alpha in enumerate(lambdas):
    for ind2, degree in enumerate(degrees):
        ridge = Ridge(alpha = alpha)
        result,_ = display_all_results(ridge, X_df, y_df, 
                                     if_display=False, if_stand=True, 
                                     poly_deg=degree)
        
        ab_ridge_scores[ind1][ind2] = result[0].r2[1]
        hf_ridge_scores[ind1][ind2] = result[1].r2[1]
        ir_ridge_scores[ind1][ind2] = result[2].r2[1]

display(pd.DataFrame(ab_ridge_scores, index=lambdas, columns=degrees))
display(pd.DataFrame(hf_ridge_scores, index=lambdas, columns=degrees))
display(pd.DataFrame(ir_ridge_scores, index=lambdas, columns=degrees))

scores_list = [ab_ridge_scores, hf_ridge_scores, ir_ridge_scores]

for ind, score in enumerate(scores_list):
    best_score = np.max(score)
    best_degree_ind = np.argmax(np.max(score, axis=0))
    best_alpha_ind = np.argmax(np.max(score, axis=1))
    best_degree = degrees[best_degree_ind]
    best_alpha = lambdas[best_alpha_ind]
    print(angs[ind] + ':')
    print("The best model has degree {} and Lambda = {}.".format(best_degree, best_alpha))                                                                      
    print("The best model has R2 = {}.\n".format(best_score))

,1,2,3,4,5
0.01,0.775065,0.780862,0.793474,0.789975,0.777323
0.05,0.775065,0.780864,0.793717,0.791744,0.785153
0.10,0.775065,0.780866,0.793864,0.792507,0.788408
0.50,0.775066,0.780881,0.794191,0.794032,0.793153
1.00,0.775067,0.780893,0.794220,0.794425,0.793836
5.00,0.775072,0.780878,0.793962,0.794663,0.793617
10.00,0.775066,0.780807,0.793710,0.794498,0.793149
50.00,0.774761,0.780222,0.791792,0.792765,0.790269
100.00,0.774124,0.779555,0.789606,0.790423,0.787027


,1,2,3,4,5
0.01,0.602819,0.636403,0.659930,0.658693,0.629463
0.05,0.602820,0.636436,0.660845,0.657979,0.641083
0.10,0.602821,0.636476,0.661426,0.657252,0.645908
0.50,0.602831,0.636750,0.662130,0.654824,0.653390
1.00,0.602843,0.637013,0.661884,0.653645,0.655513
5.00,0.602937,0.637910,0.661634,0.653029,0.657756
10.00,0.603049,0.638177,0.662741,0.654850,0.656851
50.00,0.603773,0.638196,0.667069,0.661815,0.651768
100.00,0.604359,0.638013,0.666535,0.663434,0.649848


,1,2,3,4,5
0.01,0.544877,0.638312,0.701091,0.698530,0.576513
0.05,0.544869,0.638329,0.702185,0.699159,0.604853
0.10,0.544858,0.638352,0.702645,0.698655,0.617450
0.50,0.544772,0.638583,0.703379,0.695230,0.632445
1.00,0.544665,0.638914,0.702876,0.694361,0.641207
5.00,0.543804,0.640211,0.696778,0.698839,0.671412
10.00,0.542727,0.639922,0.692507,0.701674,0.682187
50.00,0.534310,0.634709,0.680213,0.698343,0.691880
100.00,0.524884,0.630658,0.671443,0.690409,0.688081


ab:
The best model has degree 4 and Lambda = 5.
The best model has R2 = 0.7946630553302625.

hf:
The best model has degree 3 and Lambda = 50.
The best model has R2 = 0.6670693257898499.

ir:
The best model has degree 3 and Lambda = 0.5.
The best model has R2 = 0.7033793863924822.



In [157]:
lambdas = [.01,.05,.1,.5,1,5,10,50,100]
degrees = np.arange(1,6)
ab_ridge_scores = np.zeros((len(lambdas),len(degrees)))
hf_ridge_scores = np.zeros((len(lambdas),len(degrees)))
ir_ridge_scores = np.zeros((len(lambdas),len(degrees)))

for ind1, alpha in enumerate(lambdas):
    for ind2, degree in enumerate(degrees):
        ridge = Ridge(alpha = alpha)
        result,_ = display_all_results(ridge, X_df_added, y_df_added,
                                     if_display=False, if_stand=True, 
                                     poly_deg=degree)
        
        ab_ridge_scores[ind1][ind2] = result[0].r2[1]
        hf_ridge_scores[ind1][ind2] = result[1].r2[1]
        ir_ridge_scores[ind1][ind2] = result[2].r2[1]

display(pd.DataFrame(ab_ridge_scores, index=lambdas, columns=degrees))
display(pd.DataFrame(hf_ridge_scores, index=lambdas, columns=degrees))
display(pd.DataFrame(ir_ridge_scores, index=lambdas, columns=degrees))

scores_list = [ab_ridge_scores, hf_ridge_scores, ir_ridge_scores]

for ind, score in enumerate(scores_list):
    best_score = np.max(score)
    best_degree_ind = np.argmax(np.max(score, axis=0))
    best_alpha_ind = np.argmax(np.max(score, axis=1))
    best_degree = degrees[best_degree_ind]
    best_alpha = lambdas[best_alpha_ind]
    print(angs[ind] + ':')
    print("The best model has degree {} and Lambda = {}.".format(best_degree, best_alpha))                                                                      
    print("The best model has R2 = {}.\n".format(best_score))

,1,2,3,4,5
0.01,0.779339,0.790371,0.805645,0.801182,0.789101
0.05,0.779503,0.790552,0.805656,0.802608,0.794573
0.10,0.779540,0.790612,0.805617,0.803060,0.797167
0.50,0.779494,0.790641,0.805385,0.803472,0.801520
1.00,0.779436,0.790594,0.805162,0.803444,0.802353
5.00,0.779379,0.790285,0.804360,0.803391,0.802957
10.00,0.779463,0.790099,0.803971,0.803426,0.802983
50.00,0.779878,0.789866,0.802594,0.803025,0.802132
100.00,0.779916,0.789703,0.801405,0.802116,0.800854


,1,2,3,4,5
0.01,0.616444,0.656601,0.694519,0.694894,0.678224
0.05,0.616260,0.656632,0.694876,0.694959,0.684240
0.10,0.616239,0.656627,0.695027,0.694263,0.688695
0.50,0.616535,0.656638,0.694898,0.691479,0.694641
1.00,0.616860,0.656659,0.694444,0.690003,0.695454
5.00,0.618099,0.656592,0.692940,0.686613,0.694527
10.00,0.618961,0.656474,0.692459,0.685969,0.692457
50.00,0.621723,0.655843,0.690697,0.686470,0.685369
100.00,0.622843,0.655115,0.687969,0.686478,0.682851


,1,2,3,4,5
0.01,0.586467,0.646678,0.712595,0.705014,0.601489
0.05,0.586538,0.646853,0.713589,0.704072,0.627129
0.10,0.586474,0.646805,0.713577,0.702655,0.639527
0.50,0.586229,0.646481,0.711588,0.697803,0.654394
1.00,0.586093,0.646339,0.709648,0.696381,0.661132
5.00,0.585876,0.646625,0.702125,0.697882,0.681777
10.00,0.585930,0.646658,0.698443,0.699854,0.688716
50.00,0.583761,0.643512,0.690029,0.699923,0.695325
100.00,0.578028,0.638849,0.682706,0.695331,0.692741


ab:
The best model has degree 3 and Lambda = 0.05.
The best model has R2 = 0.8056564691399584.

hf:
The best model has degree 5 and Lambda = 1.
The best model has R2 = 0.6954541564287371.

ir:
The best model has degree 3 and Lambda = 0.05.
The best model has R2 = 0.7135891878438536.



## **F. LASSO**

In [46]:
lambdas = [.01,.05,.1,.5,1,5,10]
degrees = np.arange(1,6)
ab_lasso_scores = np.zeros((len(lambdas),len(degrees)))
hf_lasso_scores = np.zeros((len(lambdas),len(degrees)))
ir_lasso_scores = np.zeros((len(lambdas),len(degrees)))

for ind1, alpha in enumerate(lambdas):
    for ind2, degree in enumerate(degrees):
        lasso = Lasso(alpha=alpha)
        result,_ = display_all_results(lasso, X_df, y_df, 
                                     if_display=False, if_stand=True, 
                                     poly_deg=degree)
        
        ab_lasso_scores[ind1][ind2] = result[0].r2[1]
        hf_lasso_scores[ind1][ind2] = result[1].r2[1]
        ir_lasso_scores[ind1][ind2] = result[2].r2[1]

display(pd.DataFrame(ab_lasso_scores, index=lambdas, columns=degrees))
display(pd.DataFrame(hf_lasso_scores, index=lambdas, columns=degrees))
display(pd.DataFrame(ir_lasso_scores, index=lambdas, columns=degrees))

scores_list = [ab_lasso_scores, hf_lasso_scores, ir_lasso_scores]

for ind, score in enumerate(scores_list):
    best_score = np.max(score)
    best_degree_ind = np.argmax(np.max(score, axis=0))
    best_alpha_ind = np.argmax(np.max(score, axis=1))
    best_degree = degrees[best_degree_ind]
    best_alpha = lambdas[best_alpha_ind]
    print(angs[ind] + ':')
    print("The best model has degree {} and Lambda = {}.".format(best_degree, best_alpha))                                                                      
    print("The best model has R2 = {}.\n".format(best_score))

,1,2,3,4,5
0.01,0.775109,0.781135,0.794017,0.794750,0.792898
0.05,0.774966,0.780674,0.791327,0.792349,0.791589
0.10,0.774661,0.780156,0.788851,0.790071,0.790393
0.50,0.772185,0.776919,0.781929,0.780797,0.781533
1.00,0.767553,0.770684,0.772111,0.769390,0.770190
5.00,0.727008,0.727008,0.692852,0.692066,0.688991
10.00,0.657677,0.657677,0.524965,0.524916,0.507041


,1,2,3,4,5
0.01,0.602945,0.637375,0.658504,0.649264,0.654407
0.05,0.603184,0.637524,0.660632,0.652710,0.648170
0.10,0.603445,0.638094,0.662052,0.654081,0.647320
0.50,0.604944,0.629086,0.642175,0.645688,0.648051
1.00,0.605124,0.626448,0.631084,0.629086,0.636235
5.00,0.582814,0.582816,0.549937,0.556732,0.544786
10.00,0.498154,0.496995,0.404436,0.410607,0.376350


,1,2,3,4,5
0.01,0.544503,0.636264,0.692386,0.690612,0.672849
0.05,0.542877,0.636688,0.682427,0.689826,0.668406
0.10,0.540569,0.636454,0.676009,0.687418,0.680220
0.50,0.510870,0.631286,0.646571,0.655153,0.647349
1.00,0.484399,0.580630,0.593550,0.616546,0.623451
5.00,0.416243,0.521021,0.468887,0.473800,0.473529
10.00,0.338867,0.390363,0.272261,0.279209,0.278518


ab:
The best model has degree 4 and Lambda = 0.01.
The best model has R2 = 0.7947496261482627.

hf:
The best model has degree 3 and Lambda = 0.1.
The best model has R2 = 0.6620516783088833.

ir:
The best model has degree 3 and Lambda = 0.01.
The best model has R2 = 0.6923856081113988.



In [158]:
lambdas = [.01,.05,.1,.5,1,5,10]
degrees = np.arange(1,6)
ab_lasso_scores = np.zeros((len(lambdas),len(degrees)))
hf_lasso_scores = np.zeros((len(lambdas),len(degrees)))
ir_lasso_scores = np.zeros((len(lambdas),len(degrees)))

for ind1, alpha in enumerate(lambdas):
    for ind2, degree in enumerate(degrees):
        lasso = Lasso(alpha=alpha)
        result,_ = display_all_results(lasso, X_df_added, y_df_added,
                                     if_display=False, if_stand=True, 
                                     poly_deg=degree)
        
        ab_lasso_scores[ind1][ind2] = result[0].r2[1]
        hf_lasso_scores[ind1][ind2] = result[1].r2[1]
        ir_lasso_scores[ind1][ind2] = result[2].r2[1]

display(pd.DataFrame(ab_lasso_scores, index=lambdas, columns=degrees))
display(pd.DataFrame(hf_lasso_scores, index=lambdas, columns=degrees))
display(pd.DataFrame(ir_lasso_scores, index=lambdas, columns=degrees))

scores_list = [ab_lasso_scores, hf_lasso_scores, ir_lasso_scores]

for ind, score in enumerate(scores_list):
    best_score = np.max(score)
    best_degree_ind = np.argmax(np.max(score, axis=0))
    best_alpha_ind = np.argmax(np.max(score, axis=1))
    best_degree = degrees[best_degree_ind]
    best_alpha = lambdas[best_alpha_ind]
    print(angs[ind] + ':')
    print("The best model has degree {} and Lambda = {}.".format(best_degree, best_alpha))                                                                      
    print("The best model has R2 = {}.\n".format(best_score))

,1,2,3,4,5
0.01,0.779119,0.789635,0.803561,0.803659,0.802627
0.05,0.779186,0.789338,0.800218,0.800828,0.800438
0.10,0.779117,0.788600,0.797330,0.798258,0.799138
0.50,0.777495,0.784789,0.789073,0.787980,0.788906
1.00,0.770715,0.775967,0.777334,0.775840,0.776503
5.00,0.725592,0.725592,0.691094,0.690424,0.687228
10.00,0.656157,0.656156,0.522837,0.522790,0.504810


,1,2,3,4,5
0.01,0.618571,0.656976,0.693268,0.688790,0.690049
0.05,0.621759,0.656656,0.691952,0.690097,0.687100
0.10,0.623666,0.655164,0.686706,0.687112,0.684236
0.50,0.628017,0.645855,0.659005,0.666185,0.668133
1.00,0.628140,0.644823,0.647316,0.643878,0.650485
5.00,0.589109,0.589106,0.557178,0.563974,0.554447
10.00,0.497926,0.496965,0.404584,0.410871,0.377001


,1,2,3,4,5
0.01,0.585287,0.644917,0.698714,0.692851,0.678573
0.05,0.584840,0.650377,0.690849,0.693854,0.673450
0.10,0.581924,0.651922,0.681489,0.688877,0.679603
0.50,0.540373,0.629923,0.648876,0.659081,0.653771
1.00,0.518787,0.583766,0.605829,0.620793,0.630065
5.00,0.407538,0.513806,0.464434,0.469021,0.468557
10.00,0.332317,0.385794,0.267615,0.274663,0.273660


ab:
The best model has degree 4 and Lambda = 0.01.
The best model has R2 = 0.8036594067228682.

hf:
The best model has degree 3 and Lambda = 0.01.
The best model has R2 = 0.6932682670274342.

ir:
The best model has degree 3 and Lambda = 0.01.
The best model has R2 = 0.6987143671261664.



## **G. Decision Tree**

In [159]:
depths = list(range(1, 21))
ab_scores, hf_scores, ir_scores = [], [], []

for depth in depths:
    dt = DecisionTreeRegressor(max_depth = depth)
    result, _ = display_all_results(dt, X_df, y_df, poly_deg=5)
    ab_scores.append(result[0].r2[1])
    hf_scores.append(result[1].r2[1])
    ir_scores.append(result[2].r2[1])
    
scores_list = [ab_scores, hf_scores, ir_scores]
find_best_param(scores_list, depths, 'depth')

r2   mean_std        mae       rmse
ab Train  0.639681  10.079009  19.729670  24.346551
   Test   0.522333  10.700861  22.758666  27.706091
hf Train  0.563725  11.493524  20.711009  26.540602
   Test   0.379369  14.882129  22.828766  29.439111
ir Train  0.501876  13.460402  22.638948  29.783013
   Test   0.430273  11.378256  17.689481  24.298213

r2   mean_std        mae       rmse
ab Train  0.811509  15.472368  12.888541  17.609209
   Test   0.632528  13.997110  18.846128  24.301017
hf Train  0.692377  14.117120  17.660714  22.286417
   Test   0.535240  16.348643  20.080416  25.475476
ir Train  0.641930  14.052029  19.346855  25.251309
   Test   0.494775  13.654335  17.203533  22.881438

r2   mean_std        mae       rmse
ab Train  0.873953  13.345052  10.570488  14.399897
   Test   0.720564  13.801018  15.986107  21.191119
hf Train  0.760445  13.879898  15.460667  19.666765
   Test   0.596499  15.355888  18.598683  23.737233
ir Train  0.687894  13.724964  17.896823  23.574959
   Test   0.547752  13.622054  15.609055  21.648582

r2   mean_std        mae       rmse
ab Train  0.907112  11.708424   8.863613  12.361542
   Test   0.763346  13.400826  14.779995  19.501576
hf Train  0.808200  13.702454  13.509600  17.597686
   Test   0.610192  15.541913  17.602971  23.330999
ir Train  0.724240  14.901807  16.832647  22.159797
   Test   0.559353  13.101883  15.545916  21.369101

r2   mean_std        mae       rmse
ab Train  0.923017  10.830834   8.101531  11.253628
   Test   0.768775  13.539145  14.717024  19.276572
hf Train  0.836900  13.151763  12.334235  16.227749
   Test   0.604406  15.586443  17.579045  23.503513
ir Train  0.755743  14.679541  15.418583  20.855650
   Test   0.604190  13.211350  14.669321  20.252776

...

r2   mean_std        mae       rmse
ab Train  0.999089   0.847492   0.445374   1.224240
   Test   0.691387  17.519069  16.407749  22.269975
hf Train  0.997056   1.243823   0.601229   2.180147
   Test   0.485972  19.651031  20.058619  26.791774
ir Train  0.989826   3.149553   1.726915   4.256341
   Test   0.357499  18.437835  18.254114  25.803446

r2   mean_std        mae       rmse
ab Train  0.999500   0.637480   0.305315   0.907321
   Test   0.697066  17.314852  16.219227  22.064126
hf Train  0.998036   0.986554   0.432952   1.780898
   Test   0.468525  19.927785  20.416348  27.242665
ir Train  0.993168   2.557132   1.288790   3.487933
   Test   0.343639  19.258499  18.581088  26.080275

r2   mean_std        mae       rmse
ab Train  0.999714   0.499455   0.202389   0.686171
   Test   0.681243  17.654027  16.535641  22.633007
hf Train  0.998856   0.725951   0.280992   1.359147
   Test   0.456032  20.132108  20.582496  27.560974
ir Train  0.995663   1.949241   0.936828   2.778942
   Test   0.358326  19.021596  18.408977  25.786850

r2   mean_std        mae       rmse
ab Train  0.999847   0.405331   0.127907   0.502041
   Test   0.689802  17.676675  16.372481  22.327075
hf Train  0.999323   0.570791   0.194044   1.045817
   Test   0.460945  20.378715  20.470992  27.436247
ir Train  0.997137   1.491291   0.692388   2.257760
   Test   0.349607  18.860585  18.601303  25.961440

ab:
The best model has depth of 6.
The best model has R2 = 0.7688269891757941.

hf:
The best model has depth of 4.
The best model has R2 = 0.6101921165973727.

ir:
The best model has depth of 5.
The best model has R2 = 0.6041895435296984.



In [160]:
depths = list(range(1, 21))
ab_scores, hf_scores, ir_scores = [], [], []

for depth in depths:
    print(depth)
    dt = DecisionTreeRegressor(max_depth = depth)
    result, _ = display_all_results(dt, X_df_added, y_df_added, poly_deg=5)
    ab_scores.append(result[0].r2[1])
    hf_scores.append(result[1].r2[1])
    ir_scores.append(result[2].r2[1])
    
scores_list = [ab_scores, hf_scores, ir_scores]
find_best_param(scores_list, depths, 'depth')

1


r2   mean_std        mae       rmse
ab Train  0.638356  10.069008  19.760894  24.374538
   Test   0.520265  10.676689  22.775552  27.723879
hf Train  0.563232  11.495636  20.758679  26.580700
   Test   0.378456  14.876020  22.883147  29.474469
ir Train  0.500261  13.382972  22.677199  29.814100
   Test   0.410046  11.604096  17.887664  24.611573

2


r2   mean_std        mae       rmse
ab Train  0.810719  15.473151  12.919438  17.633934
   Test   0.630297  13.986396  18.904819  24.337709
hf Train  0.692340  14.125832  17.678504  22.308803
   Test   0.534935  16.354651  20.097220  25.495708
ir Train  0.642407  13.987175  19.348301  25.219979
   Test   0.480912  13.729902  17.286913  23.086110

3


...

19


r2   mean_std        mae       rmse
ab Train  0.999875   0.406066   0.120018   0.453115
   Test   0.746327  14.926237  14.718004  20.159990
hf Train  0.999524   0.617647   0.190529   0.877075
   Test   0.554881  17.898344  18.645560  24.942986
ir Train  0.996556   1.604086   0.831035   2.474861
   Test   0.401498  18.376620  17.108967  24.789225

20


r2   mean_std        mae       rmse
ab Train  0.999938   0.336955   0.072938   0.318350
   Test   0.752897  14.833377  14.522581  19.897201
hf Train  0.999713   0.509972   0.128308   0.681362
   Test   0.570250  17.446432  18.158508  24.508588
ir Train  0.997661   1.292750   0.633437   2.039812
   Test   0.441500  18.122585  16.868506  23.946498

ab:
The best model has depth of 6.
The best model has R2 = 0.7780312883067406.

hf:
The best model has depth of 8.
The best model has R2 = 0.6194828913274988.

ir:
The best model has depth of 3.
The best model has R2 = 0.5892061618840542.



## **H. Random Forest**

In [176]:
forest = RandomForestRegressor(n_estimators=100, max_depth=8)
display_all_results(forest, X_df, y_df, poly_deg=3);

r2   mean_std        mae       rmse
ab Train  0.964760   7.392968   5.483649   7.614031
   Test   0.794959  12.978763  13.805650  18.152329
hf Train  0.925808   8.789922   8.320427  10.944857
   Test   0.644144  14.822792  16.830980  22.291793
ir Train  0.888974  10.248660  10.401959  14.060901
   Test   0.669399  12.628248  13.505799  18.509436

In [179]:
forest = RandomForestRegressor(n_estimators=100, max_depth=8)
display_all_results(forest, X_df_added, y_df_added, poly_deg=3);

r2   mean_std        mae       rmse
ab Train  0.973134   6.360463   4.738184   6.643495
   Test   0.805563  12.404694  13.298312  17.649919
hf Train  0.950976   7.071727   6.712307   8.905242
   Test   0.704424  13.995740  15.054805  20.325673
ir Train  0.909037   9.209326   9.540736  12.719892
   Test   0.692686  12.174204  13.053333  17.763208

In [219]:
depths = list(range(1, 11))
max_features = [0.25, 0.5, 0.75, 1]
ab_lasso_scores = np.zeros((len(depths),len(max_features)))
hf_lasso_scores = np.zeros((len(depths),len(max_features)))
ir_lasso_scores = np.zeros((len(depths),len(max_features)))

for ind1, depth in enumerate(depths):
    for ind2, max_feature in enumerate(max_features):
        print('Depth =', depth, 'Max_feature =', max_feature)
        forest = RandomForestRegressor(n_estimators=100, max_depth=depth, 
                                       max_features=max_feature)
        result, _ = display_all_results(forest, X_df_added, y_df_added, poly_deg=3)
        ab_lasso_scores[ind1][ind2] = result[0].r2[1]
        hf_lasso_scores[ind1][ind2] = result[1].r2[1]
        ir_lasso_scores[ind1][ind2] = result[2].r2[1]

display(pd.DataFrame(ab_lasso_scores, index=depths, columns=max_features))
display(pd.DataFrame(hf_lasso_scores, index=depths, columns=max_features))
display(pd.DataFrame(ir_lasso_scores, index=depths, columns=max_features))
scores_list = [ab_lasso_scores, hf_lasso_scores, ir_lasso_scores]
   
        

Depth = 1 Max_feature = 0.25


r2   mean_std        mae       rmse
ab Train  0.720502   7.590537  17.118741  21.428189
   Test   0.615403   9.008633  20.228698  24.823103
hf Train  0.617175   9.964450  19.720600  24.885194
   Test   0.476940  12.902428  21.113600  27.038700
ir Train  0.534889  11.135274  23.278237  28.762630
   Test   0.574702   7.564767  15.941114  20.896671

Depth = 1 Max_feature = 0.5


r2   mean_std        mae       rmse
ab Train  0.689394   8.633887  18.186182  22.589200
   Test   0.580124   9.953207  21.253947  25.936663
hf Train  0.603440  10.539541  19.955937  25.327660
   Test   0.449659  13.493184  21.716479  27.734859
ir Train  0.540100  11.624084  22.756532  28.601041
   Test   0.557939   8.341179  16.141908  21.304500

Depth = 1 Max_feature = 0.75


...

r2   mean_std        mae       rmse
ab Train  0.969820   6.571941   5.141696   7.041340
   Test   0.792469  12.874955  13.883946  18.234537
hf Train  0.926476   7.897516   8.426953  10.905783
   Test   0.668919  13.905537  16.394564  21.511828
ir Train  0.894361   8.913779  10.593518  13.707633
   Test   0.715307  10.920192  13.065793  17.096936

,0.25,0.5,0.75,1.0
1,0.615403,0.580124,0.550978,0.420057
2,0.698724,0.693980,0.670568,0.649330
3,0.745948,0.747694,0.741868,0.728558
4,0.777155,0.782363,0.783211,0.757709
5,0.792544,0.794659,0.795931,0.772639
6,0.798803,0.801736,0.801651,0.782198
7,0.801520,0.804159,0.805860,0.787157
8,0.804048,0.806735,0.807394,0.790510
9,0.804757,0.809197,0.809461,0.793104
10,0.807480,0.810291,0.808777,0.792469


,0.25,0.5,0.75,1.0
1,0.476940,0.449659,0.428361,0.223160
2,0.600393,0.586194,0.580202,0.479777
3,0.635253,0.631293,0.631375,0.566409
4,0.647011,0.652604,0.652491,0.615361
5,0.664115,0.666923,0.671479,0.635085
6,0.670831,0.679532,0.683121,0.643092
7,0.686001,0.688637,0.698200,0.654164
8,0.693967,0.699990,0.700763,0.659644
9,0.697423,0.702816,0.707158,0.675912
10,0.703889,0.705746,0.707947,0.668919


,0.25,0.5,0.75,1.0
1,0.574702,0.557939,0.515913,0.219402
2,0.620595,0.578231,0.568139,0.470651
3,0.630099,0.629825,0.620630,0.575720
4,0.656617,0.654933,0.648367,0.604383
5,0.669122,0.655352,0.639693,0.632319
6,0.684987,0.679759,0.671229,0.666862
7,0.701605,0.695515,0.690223,0.686362
8,0.716459,0.701804,0.704410,0.701967
9,0.719040,0.713601,0.708758,0.709800
10,0.724721,0.725758,0.709261,0.715307


IndexError: list index out of range

In [221]:
for ind, score in enumerate(scores_list):
    best_score = np.max(score)
    best_degree_ind = np.argmax(np.max(score, axis=0))
    best_alpha_ind = np.argmax(np.max(score, axis=1))
    best_degree = max_features[best_degree_ind]
    best_alpha = depths[best_alpha_ind]
    print(angs[ind] + ':')
    print("The best model has depth = {} and max_feature = {}.".format(best_degree, best_alpha))                                                                      
    print("The best model has R2 = {}.\n".format(best_score))  

ab:
The best model has depth = 0.5 and max_feature = 10.
The best model has R2 = 0.8102907174628234.

hf:
The best model has depth = 0.75 and max_feature = 10.
The best model has R2 = 0.707947319008627.

ir:
The best model has depth = 0.5 and max_feature = 10.
The best model has R2 = 0.7257575157235445.



## **I. Boosting**

In [223]:
from xgboost import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
from catboost import CatBoostRegressor

In [181]:
xgb = XGBRegressor()
display_all_results(xgb, X_df, y_df, poly_deg=3);

r2   mean_std        mae       rmse
ab Train  0.945032   9.227888   6.734068   9.509303
   Test   0.792408  12.769256  13.942710  18.264920
hf Train  0.883433  10.895499  10.475998  13.718869
   Test   0.650838  14.893343  16.826315  22.081145
ir Train  0.842136  11.724480  12.670775  16.766489
   Test   0.654466  12.334764  14.068693  18.922829

In [182]:
xgb = XGBRegressor()
display_all_results(xgb, X_df_added, y_df_added, poly_deg=3);

r2   mean_std        mae       rmse
ab Train  0.957497   8.127857   5.861886   8.356155
   Test   0.806481  12.489268  13.222323  17.608188
hf Train  0.925544   8.751848   8.388052  10.974622
   Test   0.719418  13.953463  14.969674  19.803409
ir Train  0.888537  10.170857  10.769584  14.080390
   Test   0.678984  12.047663  13.891518  18.154892

In [224]:
xgb = XGBRegressor(max_depth=10, n_jobs=-1, max_features=0.5)
display_all_results(xgb, X_df_added, y_df_added, poly_deg=3);

r2   mean_std        mae       rmse
ab Train  0.999774   0.637062   0.438409   0.609184
   Test   0.807097  12.973329  13.020379  17.580171
hf Train  0.999782   0.603143   0.418862   0.593538
   Test   0.718117  14.148767  14.575026  19.849270
ir Train  0.999641   0.719508   0.559872   0.798641
   Test   0.714146  12.453013  12.309883  17.131770

In [314]:
ada = AdaBoostRegressor(base_estimator=RandomForestRegressor(n_estimators=100, max_depth=12, max_features=0.5), n_estimators=100, loss='square')
display_all_results(ada, X_df_added, y_df_added, poly_deg=3);
    

r2   mean_std        mae       rmse
ab Train  0.994896   2.416440   2.571844   2.895563
   Test   0.809607  12.662546  13.040656  17.465406
hf Train  0.990702   2.878475   3.528079   3.878184
   Test   0.746462  12.994416  14.258091  18.824854
ir Train  0.983927   3.267041   4.887371   5.346830
   Test   0.741833  11.305865  12.153426  16.280985

In [226]:
clf =  AdaBoostRegressor()
display_all_results(clf, X_df_added, y_df_added, poly_deg=3);

r2   mean_std        mae       rmse
ab Train  0.905944   8.494470  10.570224  12.430533
   Test   0.775812  12.272683  15.445112  18.952205
hf Train  0.838074   9.414935  13.645183  16.184487
   Test   0.708785  11.972230  16.142565  20.175160
ir Train  0.754347   9.979217  17.758259  20.903112
   Test   0.643759   9.233069  15.376199  19.125023

In [229]:
clf = LGBMRegressor()
display_all_results(clf, X_df_added, y_df_added, poly_deg=3);

r2   mean_std        mae       rmse
ab Train  0.989973   3.918102   3.015817   4.058717
   Test   0.808975  12.559419  12.983248  17.494399
hf Train  0.982982   4.558194   3.987053   5.246791
   Test   0.734621  13.724970  14.336482  19.259425
ir Train  0.977474   5.084236   4.725008   6.329769
   Test   0.703551  12.527984  12.749305  17.446391

In [231]:
clf = CatBoostRegressor(verbose=False)
display_all_results(clf, X_df_added, y_df_added, poly_deg=3);

r2   mean_std        mae       rmse
ab Train  0.970982   6.696114   4.898650   6.904451
   Test   0.811617  12.589948  12.969229  17.372992
hf Train  0.954713   7.236837   6.443011   8.559100
   Test   0.730316  13.753485  14.685054  19.415001
ir Train  0.946233   7.745285   7.313509   9.779302
   Test   0.735704  11.566283  12.401721  16.473098

In [232]:
clf = CatBoostRegressor(verbose=False, max_depth=10, n_estimators=100)
display_all_results(clf, X_df_added, y_df_added, poly_deg=3);

r2   mean_std        mae       rmse
ab Train  0.925584   8.796673   8.095190  11.056774
   Test   0.782788  12.174500  14.240207  18.654992
hf Train  0.898377   8.924548  10.020524  12.821414
   Test   0.700785  13.559069  15.581769  20.450417
ir Train  0.854887  10.019681  12.895308  16.065857
   Test   0.699962  10.728880  13.729617  17.551657